<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson8/NLP_intro_hw_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурирующими, выяснить какая архитектура больше подходит для задачи сентимент анализа на данных с вебинара

1. построить свёрточную архитектуру
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше


In [1]:
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.1 MB/s eta 0:00:00


In [2]:
!pip install stop_words -q

  Preparing metadata (setup.py) ... done


In [3]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
# import annoy
from gensim.models import Word2Vec, FastText

In [6]:
# Загрузка данных

df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_val = pd.read_csv("/content/val.csv")

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0.0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1.0
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0.0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1.0
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1.0


In [7]:
# Препроцессинг текста

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(text):
    text = str(text)
    text = re.sub(r'RT*', '', text)
    text = re.sub(r'@[\w]*', ' ', text)
    text = "".join(word for word in text if word not in exclude)
    text = text.lower()
    text = re.sub("не\s", "не", text)
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in sw]
    
    return " ".join(text)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

df_train.head()

,id,text,class
0,0,неуезжаааааааать ❤ нехотеть уезжать,0.0
1,1,ребята девчата кино любовь завтра вотэтолюбовь,1.0
2,2,ненавидеть пробка ретвит,0.0
3,3,хотеться котлета покиевск запретный плод,1.0
4,4,босапоп есбосой небояться мороз,1.0


In [8]:
# Импорт библиотек для построения сети

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [9]:
import tensorflow as tf

In [10]:
# Создание корпусов

text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = df_train['class'].values
y_val = df_val['class'].values

# Построим сверточную сеть.

In [12]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(256, 3, padding='same'))
model.add(Conv1D(128, 3, padding='same'))
model.add(Activation("relu"))
# model.add(MaxPooling1D())
model.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid'))
model.add(Conv1D(64, 3, padding='same'))
# model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 30)            2781660   
                                                                 
 masking (Masking)           (None, 27, 30)            0         
                                                                 
 conv1d (Conv1D)             (None, 27, 256)           23296     
                                                                 
 conv1d_1 (Conv1D)           (None, 27, 128)           98432     
                                                                 
 activation (Activation)     (None, 27, 128)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 13, 128)          0         
 )                                                               
                                                        

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [15]:
early_stopping=EarlyStopping(monitor='val_loss') 

history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 45s 173ms/step - loss: 0.6876 - accuracy: 0.5653 - val_loss: nan - val_accuracy: 0.5705
Epoch 2/10
258/258 [==============================] - 47s 182ms/step - loss: 0.6837 - accuracy: 0.5772 - val_loss: nan - val_accuracy: 0.5753


In [18]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

709/709 [==============================] - 7s 9ms/step - loss: 0.6870 - accuracy: 0.5499


Test score: 0.6870220899581909
Test accuracy: 0.5498831868171692


In [19]:
results = []

results.append(['CNN', score[0], score[1]])

# Simple RNN

In [20]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy', 
    metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') 

history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 29s 101ms/step - loss: 0.7020 - accuracy: 0.5024 - val_loss: nan - val_accuracy: 0.5299
Epoch 2/10
258/258 [==============================] - 23s 91ms/step - loss: 0.6930 - accuracy: 0.5177 - val_loss: nan - val_accuracy: 0.5529


In [21]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

709/709 [==============================] - 7s 10ms/step - loss: 0.6904 - accuracy: 0.5444


Test score: 0.6904311776161194
Test accuracy: 0.5443724393844604


In [22]:
results.append(['RNN', score[0], score[1]])

# LSTM

In [23]:
# LSTM - Long short-term memory - Нейронная сеть долгой краткосрочной памяти

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 39s 132ms/step - loss: 0.5876 - accuracy: 0.6752 - val_loss: nan - val_accuracy: 0.7294
Epoch 2/10
258/258 [==============================] - 34s 133ms/step - loss: 0.3904 - accuracy: 0.8328 - val_loss: nan - val_accuracy: 0.7315


In [24]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM', score[0], score[1]])

709/709 [==============================] - 7s 10ms/step - loss: 0.5669 - accuracy: 0.7202


Test score: 0.5668523907661438
Test accuracy: 0.7201869487762451


# GRU

In [25]:
# GRU - Gated Recurrent Units - Управляемые рекуррентные блоки

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 33s 116ms/step - loss: 0.5846 - accuracy: 0.6783 - val_loss: nan - val_accuracy: 0.7270
Epoch 2/10
258/258 [==============================] - 28s 110ms/step - loss: 0.3832 - accuracy: 0.8357 - val_loss: nan - val_accuracy: 0.7259


In [26]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU', score[0], score[1]])

709/709 [==============================] - 6s 8ms/step - loss: 0.5810 - accuracy: 0.7118


Test score: 0.5810098052024841
Test accuracy: 0.7117665410041809


# RNN + CNN

In [27]:
# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(SimpleRNN(96, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 34s 123ms/step - loss: 0.6001 - accuracy: 0.6579 - val_loss: nan - val_accuracy: 0.7260
Epoch 2/10
258/258 [==============================] - 32s 125ms/step - loss: 0.3761 - accuracy: 0.8362 - val_loss: nan - val_accuracy: 0.7208


In [28]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['RNN + CNN', score[0], score[1]])

709/709 [==============================] - 7s 9ms/step - loss: 0.5853 - accuracy: 0.7128


Test score: 0.5852973461151123
Test accuracy: 0.7127804756164551


# LSTM + CNN

In [29]:
# LSTM + CNN

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) 
model.add(LSTM(64,  recurrent_dropout=0.2, return_sequences="True")) 
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 44s 156ms/step - loss: 0.5902 - accuracy: 0.6618 - val_loss: nan - val_accuracy: 0.7289
Epoch 2/10
258/258 [==============================] - 40s 154ms/step - loss: 0.3860 - accuracy: 0.8335 - val_loss: nan - val_accuracy: 0.7243


In [30]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM + CNN', score[0], score[1]])

709/709 [==============================] - 8s 11ms/step - loss: 0.5795 - accuracy: 0.7192


Test score: 0.5795190930366516
Test accuracy: 0.71921706199646


# GRU + CNN

In [31]:
# GRU + CNN

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) 
model.add(GRU(64,  recurrent_dropout=0.2, return_sequences="True")) 
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
258/258 [==============================] - 38s 133ms/step - loss: 0.5853 - accuracy: 0.6702 - val_loss: nan - val_accuracy: 0.7344
Epoch 2/10
258/258 [==============================] - 33s 129ms/step - loss: 0.3762 - accuracy: 0.8369 - val_loss: nan - val_accuracy: 0.7307


In [32]:
score = model.evaluate(X_valid, y_val, batch_size=32, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU + CNN', score[0], score[1]])

709/709 [==============================] - 7s 9ms/step - loss: 0.5768 - accuracy: 0.7148


Test score: 0.5767656564712524
Test accuracy: 0.714808464050293


# Сводные результаты.

In [33]:
results_df = pd.DataFrame(results, columns = ['Model', 'Test score', 'Test accuracy'])
results_df

,Model,Test score,Test accuracy
0,CNN,0.687022,0.549883
1,RNN,0.690431,0.544372
2,LSTM,0.566852,0.720187
3,GRU,0.581010,0.711767
4,RNN + CNN,0.585297,0.712780
5,LSTM + CNN,0.579519,0.719217
6,GRU + CNN,0.576766,0.714808


Лучшее качество по accuracy показала модель LSTM. Впрочем качество отличается незначительно. И вероятно можно перебирать параметры и улучшить результаты моделей.